In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t

In [2]:
def family_of_loss_func(actual, predicted, degree):
    """
    Implemented from:
    Patton, A. J., 2011. Volatility forecasting comparison using imperfect 
    volatility proxies, Journal of Econometrics 160, 246-256.
    """
    if degree == -2:
        # QLIKE
        loss = actual / predicted - np.log(actual / predicted) - 1
    elif degree == -1:
        loss = predicted - actual + actual * np.log(actual / predicted)
    else:
        # MSE if degree = 0
        loss = (np.sqrt(actual) ** (2 * degree + 4) - predicted ** (degree + 2)) / ((degree + 1) * (degree + 2))
        loss -= (1 / (degree + 1)) * (predicted ** (degree + 1)) * (actual - predicted)
    return loss

def panel_DM(act, pred1, pred2, degree = 0):
    """
    Implemented from:
    Timmermann, A., Zhu, Y., 2019. Comparing Forecasting Performance with Panel Data
    """
    l1 = family_of_loss_func(act, pred1, degree)
    l2 = family_of_loss_func(act, pred2, degree)
    l1_mean = np.nanmean(l1)
    l2_mean = np.nanmean(l2)
    delta_l = l1 - l2
    n_T_sqrt = np.sqrt(np.sum(~np.isnan(delta_l)))
    sum_delta_l = np.nansum(delta_l)
    sigma_delta_l = np.nanstd(delta_l)
    j_dm = sum_delta_l / (sigma_delta_l * n_T_sqrt)
    p_value = 2 * t.cdf(-np.abs(j_dm), df = np.sum(~np.isnan(delta_l)) - 1)
    return j_dm, p_value

In [6]:
print('Daily MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/daily_mse.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Daily MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(7.984, 0.0)","(6.404, 0.0)","(6.579, 0.0)","(4.805, 0.0)","(5.788, 0.0)"
EWMA,,,"(1.637, 0.102)","(2.256, 0.024)","(-3.174, 0.002)","(0.333, 0.739)"
GARCH_NORM,,,,"(1.84, 0.066)","(-7.127, 0.0)","(-3.08, 0.002)"
GARCH_STUD,,,,,"(-7.021, 0.0)","(-3.557, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(9.109, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [7]:
print('Two-Daily MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_daily_mse.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Daily MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(10.908, 0.0)","(8.886, 0.0)","(9.086, 0.0)","(6.578, 0.0)","(7.955, 0.0)"
EWMA,,,"(1.228, 0.22)","(2.258, 0.024)","(-4.924, 0.0)","(-0.522, 0.602)"
GARCH_NORM,,,,"(3.41, 0.001)","(-7.69, 0.0)","(-3.18, 0.001)"
GARCH_STUD,,,,,"(-7.979, 0.0)","(-4.37, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(10.636, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [8]:
print('Weekly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/weekly_mse.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Weekly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(10.269, 0.0)","(9.184, 0.0)","(9.299, 0.0)","(10.346, 0.0)","(10.069, 0.0)"
EWMA,,,"(-0.886, 0.375)","(2.252, 0.024)","(-3.918, 0.0)","(0.421, 0.674)"
GARCH_NORM,,,,"(7.733, 0.0)","(-3.522, 0.0)","(1.582, 0.114)"
GARCH_STUD,,,,,"(-4.424, 0.0)","(-1.602, 0.109)"
GARCH_MIDAS_NORM,,,,,,"(6.485, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [10]:
print('Two-weekly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_weekly_mse.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-weekly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(16.056, 0.0)","(13.733, 0.0)","(14.307, 0.0)","(15.294, 0.0)","(15.277, 0.0)"
EWMA,,,"(-2.862, 0.004)","(1.858, 0.063)","(-3.409, 0.001)","(2.289, 0.022)"
GARCH_NORM,,,,"(14.732, 0.0)","(-1.866, 0.062)","(7.207, 0.0)"
GARCH_STUD,,,,,"(-4.655, 0.0)","(1.367, 0.172)"
GARCH_MIDAS_NORM,,,,,,"(9.965, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [11]:
print('Monthly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/monthly_mse.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Monthly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(19.207, 0.0)","(16.095, 0.0)","(16.997, 0.0)","(18.627, 0.0)","(18.378, 0.0)"
EWMA,,,"(-4.389, 0.0)","(0.609, 0.543)","(-2.95, 0.003)","(2.192, 0.028)"
GARCH_NORM,,,,"(17.177, 0.0)","(0.217, 0.828)","(10.729, 0.0)"
GARCH_STUD,,,,,"(-3.907, 0.0)","(2.944, 0.003)"
GARCH_MIDAS_NORM,,,,,,"(11.233, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [12]:
print('Daily QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/daily_qlike.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Daily QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(5.947, 0.0)","(22.619, 0.0)","(15.541, 0.0)","(25.077, 0.0)","(21.385, 0.0)"
EWMA,,,"(3.393, 0.001)","(3.371, 0.001)","(3.197, 0.001)","(3.085, 0.002)"
GARCH_NORM,,,,"(-3.284, 0.001)","(1.043, 0.297)","(-1.451, 0.147)"
GARCH_STUD,,,,,"(2.493, 0.013)","(1.871, 0.061)"
GARCH_MIDAS_NORM,,,,,,"(-3.784, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [13]:
print('Two-Daily QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_daily_qlike.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Daily QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(5.404, 0.0)","(24.088, 0.0)","(15.687, 0.0)","(26.122, 0.0)","(21.418, 0.0)"
EWMA,,,"(6.335, 0.0)","(5.197, 0.0)","(5.961, 0.0)","(5.141, 0.0)"
GARCH_NORM,,,,"(-9.367, 0.0)","(1.744, 0.081)","(-3.458, 0.001)"
GARCH_STUD,,,,,"(6.025, 0.0)","(3.689, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(-11.268, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [14]:
print('Weekly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/weekly_qlike.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Weekly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(6.141, 0.0)","(30.868, 0.0)","(19.925, 0.0)","(34.402, 0.0)","(27.231, 0.0)"
EWMA,,,"(16.268, 0.0)","(12.7, 0.0)","(17.39, 0.0)","(14.627, 0.0)"
GARCH_NORM,,,,"(-24.112, 0.0)","(9.412, 0.0)","(-2.777, 0.005)"
GARCH_STUD,,,,,"(18.733, 0.0)","(12.54, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(-28.742, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [15]:
print('Two-Weekly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_weekly_qlike.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Weekly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(6.671, 0.0)","(29.675, 0.0)","(21.965, 0.0)","(34.501, 0.0)","(28.514, 0.0)"
EWMA,,,"(27.769, 0.0)","(21.867, 0.0)","(31.561, 0.0)","(27.309, 0.0)"
GARCH_NORM,,,,"(-37.483, 0.0)","(16.592, 0.0)","(3.441, 0.001)"
GARCH_STUD,,,,,"(29.554, 0.0)","(23.094, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(-32.184, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [16]:
print('Monthly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/Monthly_qlike.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Monthly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(2.388, 0.017)","(29.07, 0.0)","(20.739, 0.0)","(33.303, 0.0)","(26.76, 0.0)"
EWMA,,,"(33.171, 0.0)","(25.009, 0.0)","(37.52, 0.0)","(31.311, 0.0)"
GARCH_NORM,,,,"(-55.275, 0.0)","(16.425, 0.0)","(-1.455, 0.146)"
GARCH_STUD,,,,,"(36.451, 0.0)","(26.989, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(-38.917, 0.0)"
GARCH_MIDAS_STUD,,,,,,
